 # CIFAR-10 Autoencoder

## Imports

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

from keras.models import Sequential

## Load and prepare the dataset

The [CIFAR 10](https://www.tensorflow.org/datasets/catalog/cifar10) dataset already has train and test splits and you can use those in this exercise. Here are the general steps:

* Load the train/test split from TFDS. Set `as_supervised` to `True` so it will be convenient to use the preprocessing function we provided.
* Normalize the pixel values to the range [0,1], then return `image, image` pairs for training instead of `image, label`. This is because you will check if the output image is successfully regenerated after going through your autoencoder.
* Shuffle and batch the train set. Batch the test set (no need to shuffle).


In [ ]:
# Preprocessing

def map_image(image , label):

    image = tf.cast(image , dtype = tf.float32)

    image = image / 255.0

    return image , image

# Parameters for data

BATCH_SIZE = 128

SHUFFLE_BUFFER_SIZE = 1024

train_dataset = tfds.load('cifar10' , as_supervised= True , split = 'train' )

test_dataset = tfds.load('cifar10' , as_supervised= True , split = 'test' )

# Preprocess both the datasets

train_dataset = train_dataset.map(map_image)

test_dataset = test_dataset.map(map_image)

# Shuffle and batch the dataset

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

test_dataset = test_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)



## Build the Model

In [ ]:
from keras.layers import Conv2D , UpSampling2D , MaxPooling2D , Input


inputs = Input(shape=(32, 32, 3,))

# Encoder layers

conv_1 = Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(inputs)

max_pool_1 = MaxPooling2D(pool_size=(2,2))(conv_1)

conv_2 = Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')(max_pool_1)

max_pool_2 = MaxPooling2D(pool_size=(2,2))(conv_2)

# Bottle neck layer

bottle_neck = Conv2D(filters = 256 , kernel_size = (3 , 3) , activation = 'relu' , padding = "same")(max_pool_2)

# Decoder layers

conv_3 = Conv2D(filters = 128 , kernel_size = ( 3 , 3 ) , activation = 'relu' , padding = 'same')(bottle_neck)

up_sample_1 = UpSampling2D(size = ( 2 , 2 ))(conv_3)

conv_4 = Conv2D( filters = 64 , kernel_size = ( 3 , 3 ) , activation = 'relu' , padding = 'same' )(up_sample_1)

up_sample_2 = UpSampling2D(size = ( 2 , 2))(conv_4)

decoder_output = Conv2D(filters = 3 , kernel_size= ( 3 , 3) , activation = 'sigmoid' , padding = 'same')(up_sample_2)

model = tf.keras.Model(inputs =inputs, outputs=decoder_output)


## Configure training parameters

In [ ]:

model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')

## Training

In [ ]:
train_steps = len(train_dataset) // 64
val_steps = len(test_dataset) // 64

### START CODE HERE ###
model.fit(train_dataset, steps_per_epoch=train_steps, validation_data=test_dataset, validation_steps=val_steps, epochs=100)
### END CODE HERE ###

## Model evaluation

In [ ]:
result = model.evaluate(test_dataset, steps=10)

## Save the Model



In [ ]:
# Save the model you just trained
model.save("temp_model.h5")